In [1]:
import re
import pandas as pd
from sqlalchemy import create_engine

DB_USER = 'root'
DB_PASSWORD = '123456'
DB_HOST = 'localhost'
DB_DATABASE = 'bossdata'
db_uri = f'mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_DATABASE}'
engine = create_engine(db_uri)

# 薪资数据处理

In [32]:
sql = "select salaryDesc from jobs"
df = pd.read_sql(sql, engine)

In [34]:
salary_split = df['salaryDesc'].str.split('-', expand=True)
df['salaryMin'] = df['salaryDesc'].map(lambda x: int(re.split(r'[-]', x)[0]))
df['salaryMax'] = df['salaryDesc'].map(lambda x: int(re.split(r'[-K]', x)[1]))
df['salaryMean'] = ((df['salaryMin'] + df['salaryMax']) / 2).astype(int)

df

,salaryDesc,salaryMin,salaryMax,salaryMean
0,14-15K,14,15,14
1,9-13K,9,13,11
2,14-28K·15薪,14,28,21
3,6-8K,6,8,7
4,8-12K,8,12,10
...,...,...,...,...
295,18-25K,18,25,21
296,8-10K,8,10,9
297,14-28K·13薪,14,28,21
298,15-25K,15,25,20


In [35]:
salaryDf = df['salaryMean'].value_counts()
salaryDf = salaryDf.reset_index().sort_values(by='salaryMean')
salaryDf['salaryMean'] = salaryDf['salaryMean'].map(lambda x: str(x) + 'K')

salaryDf

,salaryMean,count
26,5K,1
24,6K,2
16,7K,6
15,8K,7
19,9K,5
10,10K,13
8,11K,14
0,12K,30
6,13K,15
7,14K,15


In [26]:
salaryDf.to_json('preData/salary.json', orient='records', index=False)

# 学历统计

In [36]:
sql = "select jobDegree from jobs"
df = pd.read_sql(sql, engine)
df.head(5)

,jobDegree
0,大专
1,大专
2,本科
3,本科
4,本科


In [37]:
df = df.value_counts()
df

jobDegree
本科           264
大专            36
Name: count, dtype: int64

# 经验统计

In [115]:
sql = "select jobExperience from jobs"
df = pd.read_sql(sql, engine)
df.head(5)

,jobExperience
0,3-5年
1,1年以内
2,经验不限
3,1年以内
4,1年以内


In [116]:
df = df.value_counts()
df

jobExperience
3-5年             135
1-3年              66
5-10年             40
经验不限              39
1年以内               9
在校/应届              9
10年以上              2
Name: count, dtype: int64

In [119]:
df = df.reset_index()
df

,jobExperience,count
0,3-5年,135
1,1-3年,66
2,5-10年,40
3,经验不限,39
4,1年以内,9
5,在校/应届,9
6,10年以上,2


In [120]:
df.to_json('preData/exp.json', orient='columns')

# 地区

In [64]:
sql = "select areaDistrict from jobs"
df = pd.read_sql(sql, engine)
df

,areaDistrict
0,浦东新区
1,浦东新区
2,青浦区
3,杨浦区
4,嘉定区
...,...
295,浦东新区
296,浦东新区
297,闵行区
298,青浦区


In [65]:
import numpy as np

df = df['areaDistrict'].replace('', np.nan)

In [66]:
df.dropna(inplace=True)

In [68]:
df = df.value_counts()
df

areaDistrict
浦东新区    94
徐汇区     34
闵行区     24
杨浦区     23
长宁区     14
静安区     13
黄浦区     13
虹口区     12
松江区      9
嘉定区      8
普陀区      8
青浦区      6
宝山区      4
奉贤区      2
金山区      1
Name: count, dtype: int64

In [72]:
df.to_json('preData/district.json', orient='columns')

# 位置

In [76]:
sql = "select businessDistrict from jobs"
df = pd.read_sql(sql, engine)
df

,businessDistrict
0,陆家嘴
1,陆家嘴
2,金泽
3,五角场
4,江桥
...,...
295,张江
296,陆家嘴
297,吴泾
298,徐泾


In [78]:
df = df['businessDistrict'].replace('', np.nan)

In [79]:
df.dropna(inplace=True)

In [80]:
df = df.value_counts()
df

businessDistrict
张江     33
陆家嘴    22
漕河泾    21
五角场    18
金桥     11
       ..
万里      1
新桥      1
海湾      1
御桥      1
徐泾      1
Name: count, Length: 84, dtype: int64

In [87]:
df = df.reset_index()
df

,businessDistrict,count
0,张江,33
1,陆家嘴,22
2,漕河泾,21
3,五角场,18
4,金桥,11
...,...,...
79,万里,1
80,新桥,1
81,海湾,1
82,御桥,1


In [90]:
df = df[df['count'] >= 4]

In [93]:
df.to_json('preData/location.json')

# 词云技能

In [94]:
sql = "select skills from jobs"
df = pd.read_sql(sql, engine)
df

,skills
0,
1,
2,Java//C++//PyTorch//技术专家带教//线上面试//Linux开发/部署经验...
3,Python
4,
...,...
295,MySQL//Python//Flask
296,Python//Django//MySQL//Redis//微服务经验
297,数据开发//Python//C/C++//计算机相关专业//服务端开发经验//机器学习工程经验
298,


In [95]:
df = df['skills'].replace('', np.nan)
df.dropna(inplace=True)

In [96]:
df

2      Java//C++//PyTorch//技术专家带教//线上面试//Linux开发/部署经验...
3                                                 Python
5        Python//Kafka//Redis//MySQL//微服务经验//架构设计经验//lua
6             Python//Django//Flask//MySQL//Linux开发/部署经验
7                    Python//Java//C++//C//Django//MySQL
                             ...                        
293                             C//RAG//GPT//LLM//Python
294                      Python//PyTorch//AI//LLM//大模型语言
295                                 MySQL//Python//Flask
296                  Python//Django//MySQL//Redis//微服务经验
297      数据开发//Python//C/C++//计算机相关专业//服务端开发经验//机器学习工程经验
Name: skills, Length: 256, dtype: object

In [97]:
list_skill = df.str.split('//')
list_skill

2      [Java, C++, PyTorch, 技术专家带教, 线上面试, Linux开发/部署经...
3                                               [Python]
5      [Python, Kafka, Redis, MySQL, 微服务经验, 架构设计经验, lua]
6           [Python, Django, Flask, MySQL, Linux开发/部署经验]
7                  [Python, Java, C++, C, Django, MySQL]
                             ...                        
293                           [C, RAG, GPT, LLM, Python]
294                    [Python, PyTorch, AI, LLM, 大模型语言]
295                               [MySQL, Python, Flask]
296                [Python, Django, MySQL, Redis, 微服务经验]
297    [数据开发, Python, C/C++, 计算机相关专业, 服务端开发经验, 机器学习工程经验]
Name: skills, Length: 256, dtype: object

In [98]:
# 使用 explode() 方法将 Series 中的每个列表元素展开成独立的行
single_skill = list_skill.explode().str.strip()
single_skill

2          Java
2           C++
2       PyTorch
2        技术专家带教
2          线上面试
         ...   
297      Python
297       C/C++
297     计算机相关专业
297     服务端开发经验
297    机器学习工程经验
Name: skills, Length: 1571, dtype: object

In [101]:
df = single_skill.value_counts().head(50)

In [104]:
df = df.reset_index()
df

,skills,count
0,Python,246
1,MySQL,169
2,Django,102
3,Flask,83
4,Redis,67
5,Docker,48
6,Linux开发/部署经验,42
7,Pandas,33
8,PostgreSQL,32
9,架构设计经验,31


In [106]:
df.to_json('preData/skills.json')

# 词云福利

In [107]:
sql = "select welfareList from companies"
df = pd.read_sql(sql, engine)
df

,welfareList
0,零食下午茶//补充医疗保险//五险一金//交通补助//年终奖//加班补助//节日福利//带薪...
1,年终奖//五险一金
2,
3,五险一金//节日福利
4,年终奖//员工旅游//零食下午茶//带薪年假//节日福利//定期体检//生日福利//五险一金
...,...
295,五险一金
296,加班补助//五险一金//带薪年假//年终奖
297,餐补//年终奖//五险一金
298,带薪年假//年终奖//五险一金//绩效奖金//员工旅游//工龄奖//补充医疗保险


In [109]:
df = df['welfareList'].replace('', np.nan)
df.dropna(inplace=True)
list_skill = df.str.split('//')
single_skill = list_skill.explode().str.strip()
single_skill

0       零食下午茶
0      补充医疗保险
0        五险一金
0        交通补助
0         年终奖
        ...  
299       全勤奖
299       年终奖
299      带薪年假
299      员工旅游
299      节日福利
Name: welfareList, Length: 2202, dtype: object

In [112]:
df = single_skill.value_counts().head(50)
df = df.reset_index()
df

,welfareList,count
0,五险一金,246
1,带薪年假,206
2,年终奖,174
3,节日福利,168
4,定期体检,150
5,员工旅游,117
6,零食下午茶,114
7,团建聚餐,87
8,补充医疗保险,79
9,绩效奖金,79


In [113]:
df.to_json('preData/welfare.json')